In [ ]:
import pandas as pd

# get all boxes by group tag
data = pd.read_json("https://api.opensensemap.org/boxes?grouptag=Futurium%202021")
data.info()

In [ ]:
# initial filtering
df_meta_filter = data.query('name != "Test" & lastMeasurementAt != "NaN"')
df_meta_filter.info()

In [ ]:
import urllib.parse
from functools import reduce
from urllib.error import HTTPError

phenomenon = ["Temperatur", "rel. Luftfeuchte", "Abstand nach links", "Abstand nach rechts", "Feinstaub PM10", "Feinstaub PM25", "Beschleunigung X-Achse", "Beschleunigung Y-Achse", "Beschleunigung Z-Achse", "Geschwindigkeit"]

# merges two dataframes
def merge(df1, df2):
  return pd.merge_asof(df1, df2,
    on='createdAt',
    by=['lat', 'lon'],
    tolerance=pd.Timedelta("2s"))

# fetches measurements of phenomenon of box and returns dataframe
def get_measurements_by_phenomenon(boxId, phenomenon):
  url = "https://api.opensensemap.org/boxes/data?from-date=2021-10-16T00:00:00.000Z&boxId={}&phenomenon={}".format(boxId, urllib.parse.quote_plus(phenomenon))
  try:
    phenom_df = pd.read_csv(url, parse_dates=['createdAt'])
  except HTTPError:
    # return empty df
    return pd.DataFrame()
  phenom_df.rename(columns={"value": phenomenon}, inplace = True)
  return phenom_df.sort_values(by=['createdAt'])

# retrieves all measurements from a box and returns a filtered dataframe 
# containing all measurements merged by timestamp
def get_measurements(boxId):
  box_measurements = [get_measurements_by_phenomenon(boxId, phenom) for phenom in phenomenon]

  # filter empty measurements
  filtered = filter(lambda e: len(e) > 0, box_measurements)
  filtered_nona = [f.dropna() for f in filtered]
  if(len(filtered_nona) > 0):
    box_df = reduce(merge, filtered_nona)
    return box_df

measurements = {row['_id']: get_measurements(row['_id']) for rowindex, row in df_meta_filter.iterrows()}

In [ ]:
weather_requests_cache = {}

In [ ]:
import requests
import os

# create folder for csv files
foldername = "futurium_data"
if not os.path.exists(foldername):
    os.makedirs(foldername)

measurements_weather = {}

def get_weather(lat, lon, timestamp):
  # around 1000m accuracy
  req_lat = "%.2f" % lat
  req_lon = "%.2f" % lon
  req_time = timestamp.strftime('%Y-%m-%dT%H:00:00Z')
  url = "https://api.brightsky.dev/weather?lat={}&lon={}&date={}".format(req_lat, req_lon, req_time)
  if(url not in weather_requests_cache):
    r = requests.get(url)
    data = r.json().get('weather')[0]
    weather_requests_cache[url] = data
    return data
  else:
    return weather_requests_cache[url]

# iterate measurements and append weather information 
for boxId in measurements:
  dframe = measurements[boxId]
  if(dframe is not None):
    for label,row in dframe.iterrows():
      timestamp = row['createdAt']
      weather = get_weather(row['lat'], row['lon'], timestamp)
      dframe.loc[label,"condition"]=weather.get("condition")
      dframe.loc[label,"precipitation"]=weather.get("precipitation")
      dframe.loc[label,"sunshine"]=weather.get("sunshine")
      dframe.loc[label,"wind_direction"]=weather.get("wind_direction")
      dframe.loc[label,"wind_speed"]=weather.get("wind_speed")
      dframe.loc[label,"visibility"]=weather.get("visibility")
      dframe.loc[label,"ambient_temperature"]=weather.get("temperature")
      dframe.loc[label,"temperature_diff"]= row['Temperatur'] - weather.get("temperature")
      dframe.loc[label,"cloud_cover"]=weather.get("cloud_cover")
      dframe.loc[label,"weather_timestamp"]=weather.get("timestamp")

    # remove orphan columns and write to csv file
    dframe = dframe.drop(['sensorId_x', 'sensorId_y'], 1)
    dframe.to_csv("{}/{}.csv".format(foldername, boxId))
    measurements_weather[boxId] = dframe


In [ ]:
!pip install keplergl

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from keplergl import KeplerGl 

# adding the boxId
for key in measurements_weather.keys():
    measurements_weather[key]['boxId'] = key 

# concatenating the DataFrames
merged_df = pd.concat(measurements_weather.values())
merged_df = merged_df.fillna(-1)
merged_df.to_csv("{}/all_measurements.csv".format(foldername, boxId))

In [ ]:
# Temperature Difference
temperature_diff_config = {'config': {'mapState': {'bearing': 0,
   'dragRotate': False,
   'isSplit': False,
   'latitude': 52.53588106460044,
   'longitude': 13.476433121427846,
   'pitch': 0,
   'zoom': 9.651193962323186},
  'mapStyle': {'mapStyles': {},
   'styleType': 'dark',
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'topLayerGroups': {},
   'visibleLayerGroups': {'3d building': False,
    'border': False,
    'building': True,
    'label': True,
    'land': True,
    'road': True,
    'water': True}},
  'visState': {'animationConfig': {'currentTime': None, 'speed': 1},
   'filters': [{'animationWindow': 'free',
     'dataId': ['measurements_1'],
     'enlarged': False,
     'id': '3ek0ujb6',
     'name': ['temperature_diff'],
     'plotType': 'histogram',
     'speed': 1,
     'type': 'range',
     'value': [-1.82, 0],
     'yAxis': None},
    {'animationWindow': 'free',
     'dataId': ['measurements_2'],
     'enlarged': False,
     'id': 'x22u67coo',
     'name': ['temperature_diff'],
     'plotType': 'histogram',
     'speed': 1,
     'type': 'range',
     'value': [0, 5],
     'yAxis': None}],
   'interactionConfig': {'brush': {'enabled': False, 'size': 4.5},
    'coordinate': {'enabled': False},
    'geocoder': {'enabled': False},
    'tooltip': {'compareMode': True,
     'compareType': 'absolute',
     'enabled': True,
     'fieldsToShow': {'measurements_1': [{'format': None,
        'name': 'Temperatur'},
       {'format': None, 'name': 'ambient_temperature'},
       {'format': None, 'name': 'temperature_diff'}],
      'measurements_2': [{'format': None, 'name': 'Temperatur'},
       {'format': None, 'name': 'ambient_temperature'},
       {'format': None, 'name': 'temperature_diff'}]}}},
   'layerBlending': 'normal',
   'layers': [{'config': {'color': [23, 184, 190],
      'columns': {'altitude': None, 'lat': 'lat', 'lng': 'lon'},
      'dataId': 'measurements_1',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': True,
      'label': 'Negative Diff',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'Uber',
        'colors': ['#00939C',
         '#4BA7AF',
         '#75BBC1',
         '#9DD0D4',
         '#C1E5E6',
         '#E6FAFA'],
        'name': 'Uber Viz Sequential 4',
        'reversed': True,
        'type': 'sequential'},
       'filled': True,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'radius': 10,
       'radiusRange': [0, 50],
       'strokeColor': None,
       'strokeColorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'thickness': 2}},
     'id': '4bzph3f',
     'type': 'point',
     'visualChannels': {'colorField': {'name': 'temperature_diff',
       'type': 'real'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile'}},
    {'config': {'color': [130, 154, 227],
      'columns': {'altitude': None, 'lat': 'lat', 'lng': 'lon'},
      'dataId': 'measurements_2',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': True,
      'label': 'Positive Diff',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'ColorBrewer',
        'colors': ['#fef0d9',
         '#fdd49e',
         '#fdbb84',
         '#fc8d59',
         '#e34a33',
         '#b30000'],
        'name': 'ColorBrewer OrRd-6',
        'type': 'sequential'},
       'filled': True,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'radius': 10,
       'radiusRange': [0, 50],
       'strokeColor': None,
       'strokeColorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'thickness': 2}},
     'id': 'oounvls',
     'type': 'point',
     'visualChannels': {'colorField': {'name': 'temperature_diff',
       'type': 'real'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile'}}],
   'splitMaps': []}},
 'version': 'v1'}

temperature_diff_map = KeplerGl(height=500, data={'measurements_1': merged_df, 'measurements_2': merged_df}, config=temperature_diff_config)
temperature_diff_map

# Abstand
Die folgende Karte visualisiert den Abstand (zwischen 10 und 400cm) nach links und rechts. Beide Layer sind aktiviert und der obere (Abstand nach Rechts) ist standardmäßig zu sehen. Um den Abstand nach Links zu sehen, muss der Abstand nach Rechts ausgeblendet werden.

In [ ]:
# Abstand map

config = {'config': {'mapState': {'bearing': 0,
   'dragRotate': False,
   'isSplit': False,
   'latitude': 52.512249068879925,
   'longitude': 13.372794339415067,
   'pitch': 0,
   'zoom': 10.286536338804893},
  'mapStyle': {'mapStyles': {},
   'styleType': 'dark',
   'threeDBuildingColor': [9.665468314072013,
    17.18305478057247,
    31.1442867897876],
   'topLayerGroups': {},
   'visibleLayerGroups': {'3d building': False,
    'border': False,
    'building': True,
    'label': True,
    'land': True,
    'road': True,
    'water': True}},
  'visState': {'animationConfig': {'currentTime': None, 'speed': 1},
   'filters': [{'animationWindow': 'free',
     'dataId': ['measurements'],
     'enlarged': False,
     'id': 'l5yuvysh7',
     'name': ['Abstand nach links'],
     'plotType': 'histogram',
     'speed': 1,
     'type': 'range',
     'value': [10, 400],
     'yAxis': None},
    {'animationWindow': 'free',
     'dataId': ['measurements'],
     'enlarged': False,
     'id': 'pi97tqjb',
     'name': ['Abstand nach rechts'],
     'plotType': 'histogram',
     'speed': 1,
     'type': 'range',
     'value': [10, 400],
     'yAxis': None}],
   'interactionConfig': {'brush': {'enabled': False, 'size': 0.5},
    'coordinate': {'enabled': False},
    'geocoder': {'enabled': False},
    'tooltip': {'compareMode': False,
     'compareType': 'absolute',
     'enabled': True,
     'fieldsToShow': {'measurements': [{'format': None, 'name': 'createdAt'},
       {'format': None, 'name': 'Temperatur'},
       {'format': None, 'name': 'rel. Luftfeuchte'},
       {'format': None, 'name': 'Abstand nach links'},
       {'format': None, 'name': 'Abstand nach rechts'}]}}},
   'layerBlending': 'normal',
   'layers': [{'config': {'color': [18, 147, 154],
      'columns': {'altitude': None, 'lat': 'lat', 'lng': 'lon'},
      'dataId': 'measurements',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': True,
      'label': 'Abstand nach Rechts',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'ColorBrewer',
        'colors': ['#c51b7d',
         '#e9a3c9',
         '#fde0ef',
         '#e6f5d0',
         '#a1d76a',
         '#4d9221'],
        'name': 'ColorBrewer PiYG-6',
        'reversed': False,
        'type': 'diverging'},
       'filled': True,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'radius': 10,
       'radiusRange': [0, 50],
       'strokeColor': None,
       'strokeColorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'thickness': 2}},
     'id': 'i8jjcmk',
     'type': 'point',
     'visualChannels': {'colorField': {'name': 'Abstand nach rechts',
       'type': 'integer'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile'}},
    {'config': {'color': [18, 147, 154],
      'columns': {'altitude': None, 'lat': 'lat', 'lng': 'lon'},
      'dataId': 'measurements',
      'hidden': False,
      'highlightColor': [252, 242, 26, 255],
      'isVisible': True,
      'label': 'Abstand nach Links',
      'textLabel': [{'alignment': 'center',
        'anchor': 'start',
        'color': [255, 255, 255],
        'field': None,
        'offset': [0, 0],
        'size': 18}],
      'visConfig': {'colorRange': {'category': 'ColorBrewer',
        'colors': ['#c51b7d',
         '#e9a3c9',
         '#fde0ef',
         '#e6f5d0',
         '#a1d76a',
         '#4d9221'],
        'name': 'ColorBrewer PiYG-6',
        'reversed': False,
        'type': 'diverging'},
       'filled': True,
       'fixedRadius': False,
       'opacity': 0.8,
       'outline': False,
       'radius': 10,
       'radiusRange': [0, 50],
       'strokeColor': None,
       'strokeColorRange': {'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300'],
        'name': 'Global Warming',
        'type': 'sequential'},
       'thickness': 2}},
     'id': 'ltsbe3g',
     'type': 'point',
     'visualChannels': {'colorField': {'name': 'Abstand nach links',
       'type': 'integer'},
      'colorScale': 'quantile',
      'sizeField': None,
      'sizeScale': 'linear',
      'strokeColorField': None,
      'strokeColorScale': 'quantile'}}],
   'splitMaps': []}},
 'version': 'v1'}

abstand_map = KeplerGl(height=600, data={'measurements': merged_df}, config=config)
abstand_map

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()